In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import os

In [2]:
# Prétraiter l'image
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [3]:
train_data = datasets.ImageFolder('dataset/train', transform=transform)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True,pin_memory=True,num_workers=os.cpu_count())

In [4]:
val_data = datasets.ImageFolder('dataset/valid', transform=transform)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=False,pin_memory=True,num_workers=os.cpu_count())

In [5]:
# Charger un modèle ResNet pré-entraîné
model = models.resnet18(pretrained=True)  # Vous pouvez choisir resnet18, resnet34, resnet50, etc.

/home/sonya/Documents/projet_perso/bird_classification/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sonya/Documents/projet_perso/bird_classification/.venv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:

# Modifier la dernière couche entièrement connectée
num_classes = len(train_data.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)

In [7]:
# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [8]:
# Définir l'entraînement du modèle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [9]:
# Boucle d'entraînement
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)

    # Calculer la perte de validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_val_loss = val_loss / len(val_loader.dataset)
    accuracy = correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Val Loss: {epoch_val_loss:.4f}, Accuracy: {accuracy:.4f}')

Epoch 1/10, Loss: 3.3665, Val Loss: 1.3372, Accuracy: 0.6606
Epoch 2/10, Loss: 1.1349, Val Loss: 0.6921, Accuracy: 0.8206
Epoch 3/10, Loss: 0.7002, Val Loss: 0.5355, Accuracy: 0.8476
Epoch 4/10, Loss: 0.4927, Val Loss: 0.4898, Accuracy: 0.8743
Epoch 5/10, Loss: 0.3592, Val Loss: 0.4061, Accuracy: 0.8933
Epoch 6/10, Loss: 0.2675, Val Loss: 0.4194, Accuracy: 0.8918
Epoch 7/10, Loss: 0.2143, Val Loss: 0.4175, Accuracy: 0.8922
Epoch 8/10, Loss: 0.1691, Val Loss: 0.4359, Accuracy: 0.8895
Epoch 9/10, Loss: 0.1427, Val Loss: 0.3850, Accuracy: 0.9044
Epoch 10/10, Loss: 0.1226, Val Loss: 0.4467, Accuracy: 0.8994
